In [14]:
# %pip install openai>1.50.0 langchain>0.3.0 langgraph langchainhub langchain-openai langchain-community langchain-cli langchain_ollama tavily-python>=0.5.0 langchain_nomic nomic[local] langserve faiss-cpu tiktoken pypdf chroma jira google-search-results numexpr beautifulsoup4 scikit-learn

In [15]:
from langgraph.graph import START,  END, MessagesState
from langchain_core.runnables import RunnableConfig
from langgraph.graph import StateGraph

In [16]:
# %pip install upgrade pip
# %pip install grandalf
# %pip install -U langgraph
# !pip install -U  langchain_core

In [17]:
def node1(state:MessagesState , config:RunnableConfig):
    return {"messages": f"Hello, {state['messages']}!"}

def node2(state:MessagesState):
    return {"messages": f"Would you like to know more?"}
    

In [18]:
builder = StateGraph(MessagesState)

In [19]:
builder.add_node(node1)
builder.add_node(node2)
#everythin starts with START node
builder.add_edge(START,"node1")
builder.add_edge("node1","node2")
builder.add_edge("node2",END)

graph = builder.compile()


In [20]:
from IPython.display import display,Image
# Image(graph.get_graph().draw_mermaid_png())
print(graph.get_graph().draw_ascii())

+-----------+  
| __start__ |  
+-----------+  
      *        
      *        
      *        
  +-------+    
  | node1 |    
  +-------+    
      *        
      *        
      *        
  +-------+    
  | node2 |    
  +-------+    
      *        
      *        
      *        
 +---------+   
 | __end__ |   
 +---------+   


In [21]:
graph.invoke({"messages" : [{"role": "user", "content": " gyan"}]})

{'messages': [HumanMessage(content=' gyan', additional_kwargs={}, response_metadata={}, id='1f29b6d5-83c9-4c73-a928-f0fd41a1fdc0'),
  HumanMessage(content="Hello, [HumanMessage(content=' gyan', additional_kwargs={}, response_metadata={}, id='1f29b6d5-83c9-4c73-a928-f0fd41a1fdc0')]!", additional_kwargs={}, response_metadata={}, id='fdaf764b-3e4a-451f-a2ae-edccad7bde73'),
  HumanMessage(content='Would you like to know more?', additional_kwargs={}, response_metadata={}, id='57cd36bf-56e7-4aa0-8686-240e393e5593')]}

In [22]:
#Edges  

In [23]:
def node1(state:dict):
    print("in node1")
    return state

def node2(state:dict):
    print("in node2")
    return state



In [24]:
builder=StateGraph(dict)
builder.add_node(node1)
builder.add_node(node2)

builder.add_edge(START, "node1")
builder.add_edge("node1","node2")
builder.add_edge("node2",END)
builder.compile()

ValueError: Failed to reach https://mermaid.ink API while trying to render your graph. Status code: 204.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [25]:
print(graph.get_graph().print_ascii())

+-----------+  
| __start__ |  
+-----------+  
      *        
      *        
      *        
  +-------+    
  | node1 |    
  +-------+    
      *        
      *        
      *        
  +-------+    
  | node2 |    
  +-------+    
      *        
      *        
      *        
 +---------+   
 | __end__ |   
 +---------+   
None


In [26]:
#Conditional Edge   

In [6]:
from langgraph.graph import START, END, MessagesState, StateGraph
from langchain_core.runnables import RunnableConfig
from typing import Literal
import random

In [27]:


# Define the node functions
def node1(state: MessagesState):
    print("in node1")
    return state

def node2(state: MessagesState):
    print("in node2")
    return state

def router_function(state: MessagesState) -> Literal["node3", "node4"]:
    return random.choice(["node3", "node4"])

def node3(state: MessagesState):
    print("in node3")
    return state

def node4(state: MessagesState):
    print("in node4")
    return state

# Create the state graph
builder = StateGraph(MessagesState)

# Add nodes
builder.add_node("node1", node1)
builder.add_node("node2", node2)
builder.add_node("node3", node3)
builder.add_node("node4", node4)

# Define the edges
builder.add_edge(START, "node1")
builder.add_edge("node1", "node2")
builder.add_conditional_edges(
    "node2",
    router_function,
    {
        "node3": "node3",
        "node4": "node4"
    }
)
builder.add_edge("node3", "node4")
builder.add_edge("node4", END)

# Compile the graph
graph = builder.compile()

# Optional: Visualize the graph
print(graph.get_graph().draw_ascii())
# 
# Run the graph
initial_state = MessagesState(messages=[])
final_state = graph.invoke(initial_state, config=RunnableConfig())


       +-----------+    
       | __start__ |    
       +-----------+    
              *         
              *         
              *         
         +-------+      
         | node1 |      
         +-------+      
              *         
              *         
              *         
         +-------+      
         | node2 |      
         +-------+      
         .        .     
       ..          .    
      .             ..  
+-------+             . 
| node3 |           ..  
+-------+          .    
         *        .     
          **    ..      
            *  .        
         +-------+      
         | node4 |      
         +-------+      
              *         
              *         
              *         
        +---------+     
        | __end__ |     
        +---------+     
in node1
in node2
in node4


#State

In [7]:
from pydantic import BaseModel, Field


In [29]:
class GraphState(BaseModel):
    number :int
    result:int

In [30]:
builder = StateGraph(GraphState)

In [31]:
def node_double(state:GraphState):
    print("Current State \n\n")
    print(f"{state.number=}, {state=}")
    return {"result": 2 * state.number}
    

In [32]:
builder.add_node(node_double)
builder.add_edge(START,"node_double")
builder.add_edge("node_double",END)
graph = builder.compile()

In [33]:
print(graph.get_graph().draw_ascii())

 +-----------+   
 | __start__ |   
 +-----------+   
        *        
        *        
        *        
+-------------+  
| node_double |  
+-------------+  
        *        
        *        
        *        
  +---------+    
  | __end__ |    
  +---------+    


In [34]:
graph.invoke({"number": 100, "result":89})

Current State 


state.number=100, state=GraphState(number=100, result=89)


{'number': 100, 'result': 200}

Now with these concepts, let us first setup a useful graph

In [1]:
import os

def _set_env_from_file(var: str, file_path: str = "openai_key.txt"):
    """
    Reads an API key from a specified file and sets it as an environment variable.
    """
    if not os.environ.get(var):
        try:
            # The 'with open' statement ensures the file is closed automatically
            with open(file_path, 'r') as f:
                # Read the first line and strip any leading/trailing whitespace
                key = f.readline().strip()

            if key:
                os.environ[var] = key
                print(f"Successfully loaded {var} from {file_path}")
            else:
                print(f"Warning: {file_path} is empty.")

        except FileNotFoundError:
            print(f"Error: Key file not found at {file_path}. Please create the file.")

# --- Execution ---
# Set the environment variable OPENAI_API_KEY from the file
_set_env_from_file('OPENAI_API_KEY',file_path="../../keys/openai")


Successfully loaded OPENAI_API_KEY from ../../keys/openai


In [16]:
from openai import OpenAI
# %pip install openai

In [17]:
client=OpenAI()

In [4]:
def call_openai_api(promt:str):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": promt}],      
    )
    return response.choices[0].message.content

In [18]:
output = call_openai_api("Suggest which is better to start as a beginner, langchain or langgraph in less than 4 sentences")

In [19]:
output

"For beginners, LangChain might be a better choice as it offers a more comprehensive set of tools and documentation for building applications with language models. Its robust community and resources can help speed up the learning process. LangGraph, while more specialized, may require a deeper understanding of graph-based structures. Ultimately, if you're looking for broader versatility and support, start with LangChain."

In [20]:
def llm_suggest_concept(state: MessagesState):
    prompt = "Suggest a concept based on this theme given: "
    theme = state['messages'][-1].content
    print(f"{prompt=}, {theme=}")
    output_story = call_openai_api(prompt+theme)
    return {"messages" : f"The output concept from llm : {output_story}"} 

def editor_node(state:MessagesState):
    prompt= "Edit the story and simplify it to 3 paragraphs"
    story=state['messages'][-1].content
    print(f"{prompt=}, {story=}")
    edited_story = call_openai_api(prompt+story)
    return {"messages" : f"The edited story: {edited_story}"}

graph_builder = StateGraph(MessagesState)
graph_builder.add_node(llm_suggest_concept)
graph_builder.add_node(editor_node)

graph_builder.add_edge(START, "llm_suggest_concept")
graph_builder.add_edge("llm_suggest_concept", "editor_node")
graph_builder.add_edge("editor_node",END)

output_graph =graph_builder.compile()


    
    

In [21]:
output_graph.get_graph().print_ascii()

     +-----------+       
     | __start__ |       
     +-----------+       
            *            
            *            
            *            
+---------------------+  
| llm_suggest_concept |  
+---------------------+  
            *            
            *            
            *            
    +-------------+      
    | editor_node |      
    +-------------+      
            *            
            *            
            *            
      +---------+        
      | __end__ |        
      +---------+        


In [22]:
from IPython.display import Markdown

output = output_graph.invoke({"messages": [{"role":"user", "content": "Langraph or Langchain"}]})


prompt='Suggest a concept based on this theme given: ', theme='Langraph or Langchain'
prompt='Edit the story and simplify it to 3 paragraphs', story='The output concept from llm : Certainly! Here’s a concept based on the theme of "Langraph" or "Langchain" that could be applied in the field of natural language processing, specifically for language understanding and generation:\n\n### Concept: **LangChain - A Multi-Modal Language Processing Framework**\n\n#### Overview:\nLangChain is a robust, multi-modal framework designed to enhance the processing and understanding of different languages through a combination of natural language processing (NLP) and knowledge graph technologies. Its primary focus is to enable richer and more context-aware language interactions in applications such as chatbots, voice assistants, translation services, and content generation tools.\n\n#### Components:\n\n1. **Language Modules**:\n   - **Language Comprehension**: Tools that utilize advanced NLP techniques 

In [23]:
Markdown(output["messages"][-1].content)

The edited story: ### Concept: **LangChain - A Multi-Modal Language Processing Framework**

LangChain is an advanced framework that enhances language processing by combining natural language processing (NLP) and knowledge graph technologies. It aims to improve interactions in applications like chatbots, voice assistants, translation services, and content generation tools by enabling deeper comprehension and context-aware dialogues across multiple languages.

The framework features several key components, including language modules for comprehension and generation, which ensure accurate and culturally relevant text. It integrates knowledge graphs that provide contextual insights, allowing LangChain to connect words to broader concepts. Additionally, the unique chain of thought mechanism simulates human reasoning, creating logical responses that help tackle complex queries. User personalization and multi-modal capabilities enable customized experiences and the processing of diverse inputs, such as images and voice recordings.

LangChain can be applied in various scenarios, including customer support through intelligent chatbots, translation services that convey intent and tone, content creation tools for marketers, and interactive language tutors for e-learning. By merging language understanding with contextual knowledge, LangChain transforms human-machine communication and enhances interactions across cultures and technologies.

In [24]:
output

{'messages': [HumanMessage(content='Langraph or Langchain', additional_kwargs={}, response_metadata={}, id='fe42f32c-f890-408d-82e8-3c290561903e'),
  HumanMessage(content='The output concept from llm : Certainly! Here’s a concept based on the theme of "Langraph" or "Langchain" that could be applied in the field of natural language processing, specifically for language understanding and generation:\n\n### Concept: **LangChain - A Multi-Modal Language Processing Framework**\n\n#### Overview:\nLangChain is a robust, multi-modal framework designed to enhance the processing and understanding of different languages through a combination of natural language processing (NLP) and knowledge graph technologies. Its primary focus is to enable richer and more context-aware language interactions in applications such as chatbots, voice assistants, translation services, and content generation tools.\n\n#### Components:\n\n1. **Language Modules**:\n   - **Language Comprehension**: Tools that utilize ad